# CIVE 5699 - Programming & Data Science in CEE
```
Week 6 - Data Acquisition and Management
Dr. Matthew J. Eckelman
TA Nail Bashan
```

Contents:
* Data Formatting
* Data Cleaning
* Data Curation


## Data cleaning and formatting

In this tutorial we start with a sample air quality data provided by [PurpleAir](https://www2.purpleair.com/) for 5 different locations in Boston. Each sensor provides temperature (°F), relative humidity (%) and particulate matter concentrations (μg/m3) from **two channels** (cfa and cfb). Timestamps are given in **UTC** (usually 5 hours ahead of Boston) and raw data is given in .csv format, with data files found in the class folder on Canvas

Let's start by importing required libraries and reading the data.

In [ ]:
# Pandas and numpy for data storage and manipulation
import pandas as pd

pd.DataFrame(zip([1,2,3],['a','b','c']),columns=['column1','column2'])

In [ ]:
import numpy as np

np.array([1,2,3,4]),np.array([1,2,3,4])*2

In [ ]:
# tqdm will provide us a progress bar, which can be helpful while running long loops
from tqdm import tqdm

for k in tqdm(range(1000)):
  weather = 'cold'

In [ ]:
# Glob and os is used to interact with the database while reading/writing, especially helpful when we have big data sourced from multiple files
import glob
import os

# matplot for plotting scatter/line plots
import matplotlib.pyplot as plt

# filter warnings so we won't get annoying package update notifications
import warnings
warnings.simplefilter(action="ignore")

# timedelta is used to manipulate datetime formats
from datetime import timedelta

In [ ]:
# In this example we only have 5 .csv files but this code could handle any number, much faster than uploading manually one by one!
# First we read all the file names in sensor_data folder ending with '.csv'
file_names = glob.glob("sensor_data/*.csv")

# and initiate an empty list for these .csv folders, an empty dataframe to put them all together
csv_list = []
df = pd.DataFrame()

file_names

In [ ]:
# Create a loop to read all these file names into a dataframe (sensor_readings) and aggregate all 5 sensor_readings into the main dataframe (df)

for file in tqdm(file_names):
    sensor_readings = pd.read_csv(file)

    # .csv files came with an extra column 'Unnamed: 0' we don't need that so we can drop it
    sensor_readings = sensor_readings.drop(['Unnamed: 0'], axis=1)

    # our file names are recorded as 'sensor#.csv' we can use name[:-4] format to not consider last 4 characters (.csv)
    # take our sensor name and make it a column in sensor_readings dataframe
    sensor_readings['ids'] = [os.path.basename(file)[:-4]] * len(sensor_readings)

    #append to the main dataframe
    df = df.append(sensor_readings)

df = df.reset_index(drop=True)
df = df.drop('Unnamed: 0.1',axis=1)

In [ ]:
# Our data looks like this, let's check boxplots to identify possible outliers

df.head()

In [ ]:
# We can see that there are some extreme temperature readings (up to 2000°F)

df[['humidity','temperature']].boxplot()

In [ ]:
# Similarly, particulate matter readings also reflect some outliers

df[['cfa','cfb']].boxplot()

In [ ]:
# Also there are some cases where two channels were not in agreement under the same conditions
plt.scatter(df.cfa,df.cfb)

# We can see this better with comparing a x=y line
x = range(int(max(df.cfa)))
y = range(int(max(df.cfa)))
plt.plot(x, y,color='red', label='x=y line')

plt.xlabel('cfa readings')
plt.ylabel('cfb readings')

Under this conditions we need to follow a data cleaning procedure before any modeling or analysis. For data cleaning and formatting we will follow the steps below:

```
1. Delete missing data (missing value for both channels).
2. Delete observations with data from only one channel.
3. Delete data with abnormal temperature (T) and relative humidity (RH) readgins. (T<-200°F, or T>10000°F; RH>100%, or RH<0%).
4. Delete data if the difference between two channels is larger than 5 μg/m3 or △>61%. (△ = ((cfa-cfb)*2) / (cfa+cfb) * 100)

```
Barkjohn, Karoline K., Brett Gantt, and Andrea L. Clements. "Development and application of a United States-wide correction for PM 2.5 data collected with the PurpleAir sensor." Atmospheric Measurement Techniques 14.6 (2021): 4617-4637.

In [ ]:
# Step 1. and 2.

# As can be seen there are multiple missing data points (for example in Temperature column)
df.sort_values('temperature')

In [ ]:
# How can we drop these rows (sensor readings) with missing values? How many readings with missing data points we have?
# Python documentation will (mostly) have all the answers you want, so visit https://pandas.pydata.org/docs/

beforedrop = len(df)


####

In [ ]:
# Step 3.
# How can we drop these rows (recordings) with abnormal temperature and humidity readings? How many recordings with abnormal data points we have?

###

rhDeleted = ##
tDeleted = ##

print(f'{rhDeleted} data points are deleted due to abnormal humidity')
print(f'{tDeleted} data points are deleted due to abnormal humidity')

In [ ]:
# Step 4.

# To avoid division by 0, we will replace all zeros with a small value

df[['cfa','cfb']] = df[['cfa','cfb']].replace(0, 0.0001)
grouppedA = np.array(df.cfa)
grouppedB = np.array(df.cfb)

condition1= ##
condition2 = ##


deleted_c1c2 = ##

print(f'{deleted_c1c2} data points are deleted due to disagreement between channels')

In [ ]:
# Finally, lets convert UTC timestamps to EDT

df['date'] = pd.to_datetime(df['date'])

# EDT is 4 hours behind UTC in May, its time to use the 'timedelta' function

###

In [ ]:
# and see how is the daily variation of PM2.5 in Boston from these 5 locations
# Get the hours as a seperate column first:

df['hour'] = pd.DatetimeIndex(df['date']).hour

# and groupby hour with hourly median readings

plt.plot(df.groupby('hour').median().cfa)
plt.xlabel('hour')
plt.ylabel('PM2.5')


## Data curation and formatting


One way to access available data is through APIs (Application Programming Interfaces). APIs act as bridges between different software systems, facilitating seamless communication and data sharing, thereby simplifying integration and the utilization of their functionalities. In this tutorial, we will access 2019 US Census Data and prepare it for use in our applications.

### Preparation

APIs typically require a registered key, which may be publicly available or require purchasing a paid service. For example, the US Census Bureau offers a free API service, so the first step would be to register for your own key.

```
https://api.census.gov/data/key_signup.html
```

In [ ]:
api_key = '--'
## change this with your API key

In [ ]:
# requests library helps Python interact with web servers, JSON is the data format we are using
import requests
import json

JSON, or JavaScript Object Notation, is a simple and readable way to organize and exchange data, using a format that resembles key-value pairs and lists. JSON has become a de facto standard for data interchange on the web. Example:
```
{
  "name": "Jon Snow",
  "age": 24,
  "house": Targaryen,
  "title": "Lord Commander"
}
```

We are interested in counties of Greater Boston area (Middlesex, Norfolk, Suffolk and Essex counties) in Massachusetts. We will start by setting our initial parameters from the API's documentation: https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tiger2006se/app_a03.pdf

In [ ]:
ma_state_code = '25'

## Can you find codes for Middlesex, Norfolk, Suffolk and Essex counties?

county_codes = [###]

From this api we can collect over 10000 variables, but today suppose we just want to find median household income, total population, and white population percentage. Required keys for these variables can be found in: https://api.census.gov/data/2019/acs/acs1/variables.html

```
B03002_003E : Total white population
B03002_001E : Total population
B19013_001E : Median household income

```

In [ ]:
# Let's add one more, can you see point out any variables you are interested?

variables = 'B03002_001E,B03002_003E,B19013_001E, ####'

In [ ]:
# We start by creating an empty dataframe, and iterate over all counties to send API requests.

demographicdata=pd.DataFrame({})

for county in county_codes:
  # the format for sending this request will be available in APIs' website, this is the format that census.gov uses:
    url='https://api.census.gov/data/2019/acs/acs5?get='+variables+'&for=tract:*&in=state:'+ma_state_code+'%20county:'+county+'&key='+api_key
    # we send a request to connect this url
    response = requests.request("GET", url)
    # load all the data (in json format) as a text
    a=json.loads(response.text)
    # and read in into a temporary (temp) dataframe
    temp=pd.DataFrame(a)
    # finally combine with our main dataframe
    demographicdata=pd.concat([demographicdata,temp])


In [ ]:
# We should format this data before using

demographicdata.head()

In [21]:
# Assign first row to column names, and change column names to variables we are using (ex. 'B03002_001E':'total_pop',)


demographicdata=demographicdata.rename(columns=demographicdata.iloc[0]).drop(demographicdata.index[0])
demographicdata=demographicdata.rename(###)

demographicdata = demographicdata.reset_index(drop=True)

SyntaxError: unexpected EOF while parsing (1396565662.py, line 7)

In [ ]:
# finally, lets calculate white population percentages in each census tract
# data is recorded as string so we should change it to integer before performing a division

demographicdata['white_perc'] = ###

In [ ]:
demographicdata.head()


## Data curation and formatting 2

In this tutorial, we will access OpenStreetMap API and prepare it for use in our applications.


OpenStreetMap [OSM](https://www.openstreetmap.org) is a collaborative, open-access mapping project that allows users to create, edit, and share detailed maps and geographic data globally. OpenStreetMap provides users with access to a wide range of Points of Interest (POIs) and allows them to store geographical data. [Overpass API](https://overpass-turbo.eu/) is widely used for data acquisition from OpenStreetMap. [Wiki.openstreetmap](https://wiki.openstreetmap.org/wiki/Map_features) has all the information you need for building a query.




In [18]:

import overpy
import pandas as pd

# Define the area we want to search (min_x, min_y, max_x, max_y)
bbox = "42.335736164968644, -71.09337538026332,42.343543696646826, -71.08362188693184"
api = overpy.Overpass()

# Send a query to API
query = f"""
[out:json];
(
    node["amenity"="fast_food"]({bbox});

);
out body;
"""

# Store the results
result = api.query(query)

In [19]:
# Define an empty dictionary
restaurants_dict = {}

for node in result.nodes:
    # Extract relevant information (e.g., name) from each node
    cuisine = node.tags.get("cuisine", "N/A")
    center = (float(node.lon), float(node.lat))
    # Add information to the dictionary
    restaurants_dict[node.tags.get("name", "N/A")] = {
        "cuisine": cuisine,
        "center": center,
        # Add more fields as needed
    }


In [20]:
# We can also store data as a dataframe

pd.DataFrame.from_dict(restaurants_dict, orient='index')

,cuisine,center
Subway,sandwich,"(-71.0905736, 42.3366147)"
Papa John's,pizza,"(-71.0840056, 42.337504)"
Kigo Kitchen,asian,"(-71.0875733, 42.3390735)"
Popeyes,chicken,"(-71.0875401, 42.3390448)"
Sweet Tomatoes Pizza,pizza,"(-71.0876747, 42.3391846)"
Burger 373,burger,"(-71.0876089, 42.3391112)"
Chaat House,american,"(-71.0875008, 42.3390053)"
Five Guys,burger,"(-71.0860734, 42.3421493)"
Dunkin',donut;coffee_shop,"(-71.0892211, 42.3365119)"
Qdoba,mexican,"(-71.0908434, 42.3397769)"


### Additional exercise:
Can you determine the number of convenience stores in the area within these boundaries?
